In [1]:
import logging

logging.basicConfig(level=logging.INFO)

In [2]:
from pathlib import Path
from fast_forward import OnDiskIndex
from fast_forward.encoder import TCTColBERTQueryEncoder


ff_index = OnDiskIndex.load(
    Path("ff_msmarco-v1-passage.tct_colbert_opq.h5"),
    TCTColBERTQueryEncoder("castorini/tct_colbert-msmarco", device="cpu"),
).to_memory()

INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
100%|██████████| 8841823/8841823 [00:06<00:00, 1406975.17it/s]


In [3]:
from fast_forward import Ranking
import ir_datasets

dataset = ir_datasets.load("msmarco-passage/trec-dl-2019/judged")
sparse_top_5k = Ranking.from_file(
    Path("msmarco-passage-test2019-sparse10000.txt"),
    {q.query_id: q.text for q in dataset.queries_iter()},
).cut(5000)

In [4]:
# standard re-ranking
ff_re_ranked_5k = ff_index(sparse_top_5k)

INFO:fast_forward.index:computed scores in 4.963282166005229 seconds


In [5]:
# re-ranking with early stopping
ff_re_ranked_5k_es = ff_index(
    sparse_top_5k,
    early_stopping=10,
    early_stopping_alpha=0.2,
    early_stopping_depths=(500, 1000, 5000),
)

INFO:fast_forward.index:depth 500: 43 queries left
INFO:fast_forward.index:depth 1000: 16 queries left
INFO:fast_forward.index:depth 5000: 12 queries left
INFO:fast_forward.index:computed scores in 1.9989375830045901 seconds


In [6]:
from ir_measures import calc_aggregate, AP, RR, nDCG
from fast_forward.util import to_ir_measures

print(
    "no re-ranking:\n",
    calc_aggregate(
        [nDCG @ 10, RR(rel=2) @ 10, AP(rel=2) @ 1000],
        dataset.qrels_iter(),
        to_ir_measures(sparse_top_5k),
    ),
    "\n\nre-ranking:\n",
    calc_aggregate(
        [nDCG @ 10, RR(rel=2) @ 10, AP(rel=2) @ 1000],
        dataset.qrels_iter(),
        to_ir_measures(sparse_top_5k.interpolate(ff_re_ranked_5k, 0.2)),
    ),
    "\n\nre-ranking with early stopping:\n",
    calc_aggregate(
        [nDCG @ 10, RR(rel=2) @ 10],
        dataset.qrels_iter(),
        to_ir_measures(sparse_top_5k.interpolate(ff_re_ranked_5k_es, 0.2)),
    ),
)

no re-ranking:
 {AP(rel=2)@1000: 0.30128706043561426, RR(rel=2)@10: 0.7024178663713547, nDCG@10: 0.5058310024399073} 

re-ranking:
 {AP(rel=2)@1000: 0.4502681620775842, RR(rel=2)@10: 0.8981173864894796, nDCG@10: 0.7009168430062368} 

re-ranking with early stopping:
 {RR(rel=2)@10: 0.8981173864894796, nDCG@10: 0.7009168430062368}
